In [2]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
import os


In [3]:
def quality_metrics(y_true, y_pred, THRESHOLD = 0.5):
  ##### METRICS #####
  # Area under ROC curve
  try:
    AUC = metrics.roc_auc_score(y_true,y_pred)
  except Exception as e:
    print('All values predicited one class!')
    AUC = 0

  # Discrete metrics
  y_pred = y_pred >= THRESHOLD

  TP = np.logical_and(y_pred,y_true).sum()
  TN = np.logical_and(np.logical_not(y_pred), np.logical_not(y_true)).sum()
  FP = np.logical_and(y_pred, np.logical_not(y_true)).sum()
  FN = np.logical_and(np.logical_not(y_pred), y_true).sum()

  # According to definition:
  accuracy = (TP+TN)/len(y_pred)
  sensitivity = TP/(TP+FN)
  specificity = TN/(TN+FP)
  #accuracy = (sensitivity+specificity)/2

  return AUC, accuracy, sensitivity, specificity


In [11]:
bf_arr = []
pi_m_arr = []
path = "/home/maiki/Downloads/"
for img in os.listdir(path+"imgs"):
    print(img+"...")
    bf = cv2.resize(cv2.imread(path+"/imgs/"+img), (256,256))
    bf = np.float32((255-bf.reshape((-1,1))))
    bf = bf/max(bf)
    bf_arr.append(bf)

for img in os.listdir(path+"pi_mask"):
    print(img+"...")
    pi_m = cv2.resize(cv2.imread(path+"/pi_mask/"+img), (256,256))
    pi_m = np.round(pi_m/255).reshape((-1,1))
    pi_m_arr.append(pi_m)



c5.jpg...


g8.jpg...
e4.jpg...
d6.jpg...
g2.jpg...
e9.jpg...
a9.jpg...
b5.jpg...
h10.jpg...
g4.jpg...
b6.jpg...
h5.jpg...
h2.jpg...
d5.jpg...
e10.jpg...
b2.jpg...
e6.jpg...
c4.jpg...
g9.jpg...
h3.jpg...
a7.jpg...
d9.jpg...
e7.jpg...
h6.jpg...
a6.jpg...
b7.jpg...
a5.jpg...
f9.jpg...
g3.jpg...
g5.jpg...
d7.jpg...
h9.jpg...
h7.jpg...
f10.jpg...
c6.jpg...
e2.jpg...
e5.jpg...
f8.jpg...
a8.jpg...
d3.jpg...
d4.jpg...
h4.jpg...
b8.jpg...
e8.jpg...
g7.jpg...
g6.jpg...
f3.jpg...
f4.jpg...
e3.jpg...
c2.jpg...
f7.jpg...
f5.jpg...
f6.jpg...
d2.jpg...
b4.jpg...
a4.jpg...
a2.jpg...
b9.jpg...
f2.jpg...
c5.jpg...
g8.jpg...
e4.jpg...
d6.jpg...
g2.jpg...
e9.jpg...
a9.jpg...
b5.jpg...
h10.jpg...
g4.jpg...
b6.jpg...
h5.jpg...
h2.jpg...
d5.jpg...
e10.jpg...
b2.jpg...
e6.jpg...
c4.jpg...
g9.jpg...
h3.jpg...
a7.jpg...
d9.jpg...
e7.jpg...
h6.jpg...
a6.jpg...
b7.jpg...
a5.jpg...
f9.jpg...
g3.jpg...
g5.jpg...
d7.jpg...
h9.jpg...
h7.jpg...
f10.jpg...
c6.jpg...
e2.jpg...
e5.jpg...
f8.jpg...
a8.jpg...
d3.jpg...
d4.jpg...
h4.j

In [9]:
pi_m_arr

NameError: name 'pi_m_arr' is not defined

# Predict images, train with other images

In [12]:
for i in range(len(pi_m_arr)):
  pi_m_arr[i]=pi_m_arr[i][:,0]

In [13]:
x_tr, x_tst, y_tr, y_tst = train_test_split(bf_arr, pi_m_arr, test_size=0.3, random_state=23)


In [14]:
x_tr = np.concatenate((x_tr[:]))
y_tr = np.concatenate((y_tr[:]))
x_tst = np.concatenate((x_tst[:]))
y_tst = np.concatenate((y_tst[:]))

In [15]:

print(x_tr.shape)
print(y_tr.shape)
print(x_tst.shape)
print(y_tst.shape)


(8060928, 1)
(8060928,)
(3538944, 1)
(3538944,)


In [16]:
lr = LogisticRegression(max_iter=1000)

lr.fit(x_tr, y_tr)

y_pred = lr.predict_proba(x_tst)[::,1]

print(quality_metrics(y_tst, y_pred))

(0.9880414287152526, 0.9560148451063368, 0.8622340888643817, 0.9720537681054396)


In [18]:
from sklearn import svm

md = svm.SVC()

md.fit(x_tr, y_tr)

y_pred = md.predict_proba(x_tst)[::,1]

print(quality_metrics(y_tst, y_pred))

# Predict pixels from image

In [4]:
bf = cv2.imread("/home/maiki/Downloads/bf.JPG", cv2.IMREAD_GRAYSCALE)
pi_m = cv2.imread("/home/maiki/Downloads/pi_m.JPG", cv2.IMREAD_GRAYSCALE)


bf_neg = 255-bf

trainingData = np.float32(bf_neg.reshape((-1,1)))
labels = np.round(pi_m/255).reshape((-1,1))
trainingData = trainingData/max(trainingData)

x_tr, x_tst, y_tr, y_tst = train_test_split(trainingData, labels, test_size=0.3, random_state=23)


In [5]:
y_tr = y_tr[:,0]
y_tst = y_tst[:,0]

print(x_tr.shape)
print(y_tr.shape)
print(x_tst.shape)
print(y_tst.shape)


(2936012, 1)
(2936012,)
(1258292, 1)
(1258292,)


In [7]:
lr = LogisticRegression(max_iter=1000)

lr.fit(x_tr, y_tr)

y_pred = lr.predict_proba(x_tst)[::,1]

print(quality_metrics(y_tst, y_pred))


(0.9954066815157586, 0.9876721778410734, 0.9202734307979881, 0.9941762936897793)


In [8]:
dt = DecisionTreeClassifier()

dt.fit(x_tr, y_tr)

y_pred = dt.predict_proba(x_tst)[::,1]

print(quality_metrics(y_tst, y_pred))

(0.9954407627384795, 0.9878517863898046, 0.9069450339079473, 0.9956594521724961)
